In [25]:
from bs4 import BeautifulSoup

In [26]:
from gensim.models import Doc2Vec

In [27]:
from gensim.models.doc2vec import TaggedDocument

In [28]:
BASE_URL = 'http://composingprograms.com/'

In [29]:
import requests

In [30]:
homesoup = BeautifulSoup(requests.get(BASE_URL).text, 'html.parser')

In [31]:
homesoup

<!DOCTYPE html>

<html lang="en">
<head>
<title>Composing Programs</title>
<meta charset="utf-8"/>
<link href="./theme/css/cp.css" rel="stylesheet" type="text/css"/>
</head>
<body class="home" id="index">
<div class="container">
<div class="nav-main">
<div class="wrap">
<a class="nav-home" href=".">
<span class="nav-logo">c<span class="nav-logo-compose">â¬</span>mp<span class="nav-logo-compose">â¬</span>sing pr<span class="nav-logo-compose">â¬</span>grams</span>
</a>
<ul class="nav-site">
<li><a href="./">Text</a></li>
<li><a href="./projects.html">Projects</a></li>
<li><a href="./tutor.html">Tutor</a></li>
<li><a href="./about.html">About</a></li>
</ul>
</div>
</div>
<section class="content wrap documentationContent">
<div class="nav-docs">
<div class="nav-docs-section">
<h3>Main</h3>
<ul>
<li><a href="./">Text</a></li>
<li><a href="./projects.html">Projects</a></li>
<li><a href="./tutor.html">Tutor</a></li>
<li><a href="./about.html">About</a></li>
</ul>
</div>
<div class="nav-doc

In [32]:
pages = [a for a in homesoup.find_all('a') if './pages/' in a['href']] # generalize later

In [33]:
pages[0]['href'].replace('./', BASE_URL)

'http://composingprograms.com/pages/11-getting-started.html'

In [34]:
def scrape(page_url):
    soup = BeautifulSoup(requests.get(page_url).text, 'html.parser')
    documents = []
    links = []
    for div in soup.find_all('div', {'class': 'section'}):
        documents.append(div.get_text())
        links.append('{0}#{1}'.format(page_url, div['id']))
    return documents, links

In [35]:
def get_link(page):
    return page['href'].replace('./', BASE_URL)

In [36]:
documents = []
links = []
for i, page in enumerate(pages):
    running = True
    while running:
        try:
            link = get_link(page)
            print(link)
            new_documents, new_links = scrape(link)
            documents.extend(new_documents)
            links.extend(new_links)
            running = False
        except:
            print('Trying page', i, 'again')

http://composingprograms.com/pages/11-getting-started.html
http://composingprograms.com/pages/12-elements-of-programming.html
http://composingprograms.com/pages/13-defining-new-functions.html
http://composingprograms.com/pages/14-designing-functions.html
http://composingprograms.com/pages/15-control.html
http://composingprograms.com/pages/16-higher-order-functions.html
Trying page 5 again
http://composingprograms.com/pages/16-higher-order-functions.html
http://composingprograms.com/pages/17-recursive-functions.html
http://composingprograms.com/pages/21-introduction.html
http://composingprograms.com/pages/22-data-abstraction.html
http://composingprograms.com/pages/23-sequences.html
http://composingprograms.com/pages/24-mutable-data.html
http://composingprograms.com/pages/25-object-oriented-programming.html
http://composingprograms.com/pages/26-implementing-classes-and-objects.html
http://composingprograms.com/pages/27-object-abstraction.html
Trying page 13 again
http://composingprograms

In [37]:
links, len(links)

(['http://composingprograms.com/pages/11-getting-started.html#getting-started',
  'http://composingprograms.com/pages/11-getting-started.html#programming-in-python',
  'http://composingprograms.com/pages/11-getting-started.html#installing-python-3',
  'http://composingprograms.com/pages/11-getting-started.html#interactive-sessions',
  'http://composingprograms.com/pages/11-getting-started.html#first-example',
  'http://composingprograms.com/pages/11-getting-started.html#errors',
  'http://composingprograms.com/pages/12-elements-of-programming.html#elements-of-programming',
  'http://composingprograms.com/pages/12-elements-of-programming.html#expressions',
  'http://composingprograms.com/pages/12-elements-of-programming.html#call-expressions',
  'http://composingprograms.com/pages/12-elements-of-programming.html#importing-library-functions',
  'http://composingprograms.com/pages/12-elements-of-programming.html#names-and-the-environment',
  'http://composingprograms.com/pages/12-elements

In [38]:
documents[0]

"\n1.1Â\xa0Â\xa0Â\xa0Getting Started\nComputer science is a tremendously broad academic discipline. The areas of\nglobally distributed systems, artificial intelligence, robotics, graphics,\nsecurity, scientific computing, computer architecture, and dozens of emerging\nsub-fields all expand with new techniques and discoveries every year. The\nrapid progress of computer science has left few aspects of human life\nunaffected. Commerce, communication, science, art, leisure, and politics have\nall been reinvented as computational domains.\nThe high productivity of computer science is only possible because the\ndiscipline is built upon an elegant and powerful set of fundamental ideas. All\ncomputing begins with representing information, specifying logic to process it,\nand designing abstractions that manage the complexity of that logic. Mastering\nthese fundamentals will require us to understand precisely how computers\ninterpret computer programs and carry out computational processes.\nThes

In [39]:
import re
import nltk
from nltk.corpus import stopwords
 
sw = set(stopwords.words('english'))

def get_words(document):
    return [word for word in re.split('\s+', re.sub('[^a-zA-Z]', ' ', document)) if word not in sw]

In [40]:
def generate_tagged_documents(documents):
    return iter([TaggedDocument(get_words(document), [link]) for link, document in zip(links, documents)])
        

In [41]:
it = generate_tagged_documents(documents)

In [42]:
model = Doc2Vec(alpha=.025, min_alpha=.025, min_count=1)

In [43]:
model.build_vocab(it)

In [44]:
model.train(it, total_examples=model.corpus_count, epochs=model.epochs)

In [23]:
model.save('textbook.doc2vec')

In [46]:
new_doc_vector = model.infer_vector(para.split(' '))

In [24]:
model = Doc2Vec.load('textbook.doc2vec')

In [47]:
import numpy as np

def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [48]:
arg_best, best = -1, -1

try:
    for i, vector in enumerate(model.docvecs):
        sim = cosine_similarity(vector, new_doc_vector)
        if sim > best:
            arg_best, best = i, sim
except:
    pass
arg_best, best

(71, 0.28020421)

In [49]:
links[71]

'http://composingprograms.com/pages/24-mutable-data.html#propagating-constraints'

In [45]:
para = 'Lists are immutable data types'